In [3]:
import pandas as pd
import numpy as np

# Load the datasets
routes = pd.read_csv('Bus_Routes.csv')
stops = pd.read_csv('Bus_Stops.csv')
mapping = pd.read_csv('Route_Stop_Mapping.csv')
ridership = pd.read_csv('Train_Ridership_2022_to_2025H1.csv')
traffic = pd.read_csv('Train_Traffic_2022_to_2025H1.csv')

# Let's use the ridership data as our base frame, because it contains Route_ID, Stop_ID, and Date together.
df = ridership.copy()

In [4]:
# Convert Date string to DateTime format
df['Date'] = pd.to_datetime(df['Date'])
traffic['Date'] = pd.to_datetime(traffic['Date'])

# Create Temporal Features
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['DayOfWeek'] = df['Date'].dt.dayofweek
df['Is_Weekend'] = df['DayOfWeek'].apply(lambda x: 1 if x >= 5 else 0) # 5=Sat, 6=Sun

In [5]:
# Calculate Total Activity (How busy the stop is in total)
df['Total_Activity'] = df['Boarding_Count'] + df['Alighting_Count']

# Calculate Net Ridership (Are people mostly getting on or getting off?)
df['Net_Ridership'] = df['Boarding_Count'] - df['Alighting_Count']

In [6]:
# 1. Merge Traffic Data (Matches on Date)
df = pd.merge(df, traffic, on='Date', how='left')

# 2. Merge Route Data (Matches on Route_ID)
routes_subset = routes[['Route_ID', 'Route_Type', 'Route_Length_km', 'Avg_Travel_Time_Min']]
df = pd.merge(df, routes_subset, on='Route_ID', how='left')

# 3. Merge Stop Data (Matches on Stop_ID)
stops_subset = stops[['Stop_ID', 'Latitude', 'Longitude', 'Stop_Type', 'Zone']]
df = pd.merge(df, stops_subset, on='Stop_ID', how='left')
df.rename(columns={'Stop_Type': 'Stop_Facility_Type'}, inplace=True) # Rename to prevent confusion with Route_Type

# 4. Merge Route-Stop Mapping (Matches on Route_ID and Stop_ID)
mapping_subset = mapping[['Route_ID', 'Stop_ID', 'Stop_Sequence', 'Dwell_Time_Min']]
df = pd.merge(df, mapping_subset, on=['Route_ID', 'Stop_ID'], how='left')

In [7]:
# Force Route_ID, Stop_ID, Date to the front of the dataset
front_cols = ['Route_ID', 'Stop_ID', 'Date']
remaining_cols = [col for col in df.columns if col not in front_cols]
df = df[front_cols + remaining_cols]

# Save to the separate CSV file
output_file = 'engineered_features_dataset.csv'
df.to_csv(output_file, index=False)

print(f"Data merged successfully! Shape of new dataset: {df.shape}")

Data merged successfully! Shape of new dataset: (195381, 24)


In [10]:
df.columns


Index(['Route_ID', 'Stop_ID', 'Date', 'Ridership_ID', 'Boarding_Count',
       'Alighting_Count', 'Year', 'Month', 'Day', 'DayOfWeek', 'Is_Weekend',
       'Total_Activity', 'Net_Ridership', 'Congestion_Level', 'Avg_Speed_kmph',
       'Route_Type', 'Route_Length_km', 'Avg_Travel_Time_Min', 'Latitude',
       'Longitude', 'Stop_Facility_Type', 'Zone', 'Stop_Sequence',
       'Dwell_Time_Min'],
      dtype='object')